In [17]:
import pandas as pd
import numpy as np
import requests
import googlemaps
from sklearn.preprocessing import MinMaxScaler

In [31]:
%run -i ../notebooks/functions.py

In [2]:
df = pd.read_csv("../raw_data/traffic/traffic.csv")

df = df[df["Date"].str.contains("2020") == True]
df['traffic'] = (df.iloc[:, 5:].sum(axis=1)) / 24
df = df[['ID', 'Roadway Name', 'To', 'traffic']].copy()

/tmp/ipykernel_32215/3097917546.py:4: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df['traffic'] = (df.iloc[:, 5:].sum(axis=1)) / 24


In [3]:
df_mean = df.groupby(['ID']).mean()
df_mean = df_mean.reset_index()

In [4]:
df = df.drop(['traffic'], axis=1)
df = df.drop_duplicates()

In [5]:
traffic = df.set_index('ID').join(df_mean.set_index('ID'))
traffic = traffic.reset_index()
traffic = traffic.drop_duplicates(subset=['ID'], keep='first')

In [6]:
gmaps = googlemaps.Client(key='GOOGLE_API_KEY')

ValueError: Invalid API key provided.

In [ ]:
lat_long = []
for i in range(len(traffic)):
    first = traffic.iloc[i][1]
    second = traffic.iloc[i][2]
    geocode_result = gmaps.geocode(f'{first}, {second}, NY')
    coords = (geocode_result[0]['geometry']['location']['lat'], geocode_result[0]['geometry']['location']['lng'])
    lat_long.append(coords)

In [ ]:
lat = []
lng = []
for i in range(349):
    lat.append(lat_long[i][0])
    lng.append(lat_long[i][1])

In [ ]:
traffic = traffic.drop(['Roadway Name', 'To', 'ID'], axis=1)
traffic['latitude'] = lat
traffic['longitude'] = lng

In [9]:
traffic

,ID,Roadway Name,To,traffic
0,1,1st AVENUE,East 61st Street,954.828704
1,2,1 Avenue,E 124th St,1027.875000
2,3,1st AVENUE,East 12th Street,662.504630
3,4,1st AVENUE,East 27th Street,684.055556
4,5,10TH AVENUE,West 46th Street,976.597222
...,...,...,...,...
368,369,YORK AVENUE,East 61st Street,337.578704
370,370,YORK AVENUE,East 75th Street,303.451389
371,371,7th AVENUE,East 28th Street,752.300926
372,372,GERARD AVENUE,East 151st Street,165.064815


In [ ]:
traffic.to_csv('../processed_data/geocoded_traffic.csv', index=False, encoding='utf-8')

In [27]:
df_car_raw = pd.read_csv("../processed_data/geocoded_traffic.csv")
df_car = np.array(df_car_raw['traffic'])
df_car = np.reshape(df_car, (349,1))

scaler = MinMaxScaler()
car = scaler.fit_transform(df_car)

In [29]:
df_ped_raw = pd.read_csv("../processed_data/geocoded_pedestrian.csv")
df_ped = np.array(df_ped_raw['ped_traffic'])
df_ped = np.reshape(df_ped, (112,1))

scaler = MinMaxScaler()
ped = scaler.fit_transform(df_ped)

In [40]:
busy = pd.DataFrame(car)

In [42]:
busy

,0
0,0.790442
1,0.851866
2,0.544628
3,0.562750
4,0.808747
...,...
344,0.271399
345,0.242702
346,0.620137
347,0.126333
